In [1]:
import pandas as pd
import numpy as np

import altair as alt
alt.data_transformers.disable_max_rows()
#import vega_datasets.data as data

from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
    RandomizedSearchCV
)

from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer,
    PolynomialFeatures
)

from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import StackingClassifier


import numpy as np
import pandas as pd
from torchvision import transforms, datasets, utils
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
import torch.utils.data as data
from torch import nn

import time

In [3]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 64 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

In [5]:
df

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.500000,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.500000,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75752,6,State_11,Commercial,Office_Uncategorized,20410.0,1995.0,8.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,132.918411,75752
75753,6,State_11,Residential,5plus_Unit_Building,40489.0,1910.0,98.0,36.6,28,43.451613,...,25,3,0,0,NaN,NaN,NaN,NaN,39.483672,75753
75754,6,State_11,Commercial,Commercial_Other,28072.0,1917.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,48.404398,75754
75755,6,State_11,Commercial,Commercial_Other,53575.0,2012.0,NaN,36.6,26,36.612903,...,6,0,0,0,NaN,NaN,NaN,NaN,592.022750,75755


In [6]:
TARGET_COLUMN = "site_eui"

### Feature Engineering/Selection

In [21]:
# Do something about missing values in direction_max_wind_speed and direction_peak_wind_speed

In [18]:
train_path = "data/train.csv"
test_path = "data/test.csv"

In [19]:
class Dataprep(Dataset):

    def __init__(self, train_path, test_path, size=0.1, is_train=True):
        df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)

        TARGET_COLUMN = "site_eui"

        self.is_train = is_train

        train_df, val_df = train_test_split(
            df,
            test_size=0.1,
            random_state=123
        )

        self.X_train, self.y_train = train_df.drop(columns=[TARGET_COLUMN]), train_df[TARGET_COLUMN]
        self.X_val, self.y_val = val_df.drop(columns=[TARGET_COLUMN]), val_df[TARGET_COLUMN]
        self.X_test = test_df

    def engineer_features(self):
        # Find Standard Deviation of min, max and avg temp among months
        min_temps = [
            "january_min_temp",
            "february_min_temp",
            "march_min_temp",
            "april_min_temp",
            "may_min_temp",
            "june_min_temp",
            "july_min_temp",
            "august_min_temp",
            "september_min_temp",
            "october_min_temp",
            "november_min_temp",
            "december_min_temp"
        ]

        max_temps = [
            "january_max_temp",
            "february_max_temp",
            "march_max_temp",
            "april_max_temp",
            "may_max_temp",
            "june_max_temp",
            "july_max_temp",
            "august_max_temp",
            "september_max_temp",
            "october_max_temp",
            "november_max_temp",
            "december_max_temp"
        ]

        avg_temps = [
            "january_avg_temp",
            "february_avg_temp",
            "march_avg_temp",
            "april_avg_temp",
            "may_avg_temp",
            "june_avg_temp",
            "july_avg_temp",
            "august_avg_temp",
            "september_avg_temp",
            "october_avg_temp",
            "november_avg_temp",
            "december_avg_temp",
        ]

        self.numeric_features = [
            "floor_area",
            "year_built",
            "energy_star_rating",
            "ELEVATION",
            "cooling_degree_days",
            "heating_degree_days",
            "precipitation_inches",
            "snowfall_inches",
            "snowdepth_inches",
            "avg_temp",
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
            "max_wind_speed",
            "days_with_fog",
            "building_age",
            "min_temp_std",
            "max_temp_std",
            "avg_temp_std",
            "0-10",
            "10-20",
            "20-30",
            "30-80",
            "80-90",
            "90-100",
            "100-110"
        ] + min_temps + max_temps + avg_temps

        self.categorical_features = [
            "Year_Factor",
            "State_Factor",
            "building_class",
            "facility_type",
            "direction_max_wind_speed",
            "direction_peak_wind_speed"
        ]

        self.drop_columns = [
            "id"
        ]
        
        for X in [self.X_train, self.X_val, self.X_test]:
            X["building_age"] = 2022 - X["year_built"]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["min_temp_std"] = X[min_temps].T.std()
            X["max_temp_std"] = X[max_temps].T.std()
            X["avg_temp_std"] = X[avg_temps].T.std()

        days_above_below = [
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
        ]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["0-10"] = X["days_below_10F"] - X["days_below_0F"]
            X["10-20"] = X["days_below_20F"] - X["days_below_10F"]
            X["20-30"] = X["days_below_30F"] - X["days_below_20F"]
            X["80-90"] = X["days_above_80F"] - X["days_above_90F"]
            X["90-100"] = X["days_above_90F"] - X["days_above_100F"]
            X["100-110"] = X["days_above_100F"] - X["days_above_110F"]
            X["30-80"] = (366 - X[days_above_below].sum(axis=1)).clip(lower=0)

    def preprocess(self, is_sklearn=False):    
        pipe_numeric_feats = make_pipeline(
           SimpleImputer(strategy="mean"),
           StandardScaler()
        )
        pipe_cat_feats = make_pipeline(
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown="ignore", sparse=False)
        )
        self.column_transformer = make_column_transformer(
            (pipe_numeric_feats, self.numeric_features),
            (pipe_cat_feats, self.categorical_features)
        )
        
        self.X_train_raw = self.X_train
        self.X_val_raw = self.X_val
        self.X_test_raw = self.X_test
        self.X_train = self.column_transformer.fit_transform(self.X_train)
        self.X_val = self.column_transformer.transform(self.X_val)
        self.X_test = self.column_transformer.transform(self.X_test)

        if not is_sklearn:
            self.X_train = torch.tensor(self.X_train, dtype=torch.float32)
            self.y_train = torch.tensor(self.y_train.values, dtype=torch.float32)
            self.X_val = torch.tensor(self.X_val, dtype=torch.float32)
            self.y_val = torch.tensor(self.y_val.values, dtype=torch.float32)
            self.X_test = torch.tensor(self.X_test, dtype=torch.float32)

## Model

In [20]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score=False):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_val": model.score(X_val, y_val),
        "mse_val": mean_squared_error(y_val, y_val_pred),
        "mape_val": mean_absolute_percentage_error(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mean_squared_error(y_train, y_train_pred)
        score_dict["mape_train"] = mean_absolute_percentage_error(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [21]:
#from fancyimpute import IterativeImputer, KNN
#imp_num = SimpleImputer(strategy="mean")
#imp_cat = SimpleImputer(strategy="most_frequent")

# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=2)
# imputer.fit_transform(X_train_)
# pipe_numeric_feats = make_pipeline(
#     mice_imp,
#     StandardScaler()
# )

# pipe_cat_feats = make_pipeline(
#     knn,
#     OneHotEncoder()
# )

# column_transformer = make_column_transformer(
#     (pipe_numeric_feats, numeric_features),
#     (OneHotEncoder(), categorical_features),
#     ("drop", drop_columns)
# )
# X_transformed = column_transformer.fit_transform(X_train)
# X_transformed

In [22]:
feature_set = Dataprep(train_path, test_path)
feature_set.engineer_features()
feature_set.preprocess(is_sklearn=True)

In [23]:
pipe_lr = make_pipeline(feature_set.column_transformer, Ridge(max_iter=10000))
pipe_rf = make_pipeline(feature_set.column_transformer, RandomForestRegressor())
pipe_xgb = make_pipeline(feature_set.column_transformer, XGBRegressor(verbosity=0))
pipe_lgbm = make_pipeline(feature_set.column_transformer, LGBMRegressor())
pipe_catboost = make_pipeline(feature_set.column_transformer, CatBoostRegressor(verbose=False))

In [24]:
models = {
    "Ridge": pipe_lr,
    "Random Forest": pipe_rf,
    "XGB": pipe_xgb,
    "LGBM": pipe_lgbm,
    "Cat Boost": pipe_catboost
}

In [27]:
results = {}

for name, model in models.items():
    print(f"Start {name}!")
    _, results[name] = cross_val_scores(
        model,
        feature_set.X_train_raw, 
        feature_set.y_train,
        feature_set.X_val_raw,
        feature_set.y_val,
        return_train_score=True
    )

    print(f"Done {name}!") 


Start Ridge!
Done Ridge!
Start Random Forest!
Done Random Forest!
Start XGB!
Done XGB!
Start LGBM!
Done LGBM!
Start Cat Boost!
Done Cat Boost!


In [28]:
pd.DataFrame(results)

,Ridge,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.385052,0.544429,0.544609,0.492902,0.530885
mse_val,1909.311986,1414.471916,1413.912253,1574.456126,1456.525966
mape_val,0.515557,0.375215,0.424664,0.453599,0.426800
r2_train,0.356391,0.937019,0.668918,0.540934,0.635735
mse_train,2204.814131,215.755881,1134.188983,1572.623749,1247.862146
mape_train,0.592824,0.158359,0.440948,0.495500,0.457658


In [29]:
# Adapted from UBC Lecture Notes

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer
}

In [30]:
# Hyperparam Tune Random Forest 

params = {
    'randomforestregressor__n_estimators': [10, 100, 1000],
    'randomforestregressor__max_depth': [5, 10, 12],
    'randomforestregressor__max_features': ['auto', 'sqrt']
}


In [31]:
pipe_rf = make_pipeline(feature_set.column_transformer, RandomForestRegressor())

random_search = RandomizedSearchCV(
    pipe_rf,
    params,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [32]:
random_search.fit(feature_set.X_train_raw, feature_set.y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                                                'ELEVATION',

In [95]:
#pd.DataFrame(random_search.cv_results)

In [35]:
random_search.best_params_

{'randomforestregressor__n_estimators': 1000,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__max_depth': 12}

In [36]:
random_search.best_score_

0.4318005845886447

In [44]:
# HyperparamTune XGBoost
params_xgb = {
    'xgbregressor__n_estimators': [10, 100, 1000],
    'xgbregressor__max_depth': [3, 5, 7, 12],
    'xgbregressor__eta': [0.01, 0.03, 0.01, 0.3]
}

In [45]:
random_search = RandomizedSearchCV(
    pipe_xgb,
    params_xgb,
    n_jobs=-1,
    n_iter=20,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [46]:
random_search.fit(feature_set.X_train_raw, feature_set.y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                                                'ELEVATION',

In [51]:
random_search.best_params_

{'xgbregressor__n_estimators': 1000,
 'xgbregressor__max_depth': 7,
 'xgbregressor__eta': 0.01}

In [52]:
random_search.best_score_

0.5659283801380433

In [ ]:
#pd.DataFrame(results_xgb.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xgbregressor__n_estimators,param_xgbregressor__max_depth,param_xgbregressor__eta,params,split0_test_neg RMSE,split1_test_neg RMSE,...,mean_test_mape,std_test_mape,rank_test_mape,split0_train_mape,split1_train_mape,split2_train_mape,split3_train_mape,split4_train_mape,mean_train_mape,std_train_mape
0,32.165711,0.467734,0.145800,0.014685,100,3,0.3,"{'xgbregressor__n_estimators': 100, 'xgbregres...",-44.906015,-42.197639,...,-52.600305,1.712084,7,-52.212735,-51.615945,-51.643463,-50.872726,-51.553715,-51.579717,0.425817
1,6.793386,0.853771,0.125048,0.034139,10,5,0.3,"{'xgbregressor__n_estimators': 10, 'xgbregress...",-47.238368,-44.167912,...,-58.349321,2.210983,8,-57.865903,-57.902502,-58.031283,-57.039269,-58.289241,-57.825640,0.420267
2,761.857459,4.299093,0.263911,0.030418,1000,7,0.3,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-38.766614,-37.586919,...,-40.787729,1.802297,2,-15.854925,-15.598037,-15.804246,-15.881822,-15.973886,-15.822583,0.125064
3,5.728950,0.161968,0.096310,0.014663,10,5,0.03,"{'xgbregressor__n_estimators': 10, 'xgbregress...",-82.405178,-79.960568,...,-72.812074,0.357883,10,-72.797749,-72.739718,-72.857839,-72.717825,-72.827175,-72.788061,0.052465
4,480.629719,8.966158,0.204403,0.035396,1000,5,0.3,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-40.214539,-38.136232,...,-43.324679,1.683031,3,-28.844111,-28.606951,-28.707926,-28.353106,-28.605490,-28.623517,0.160940
5,714.549261,10.710977,0.221960,0.013096,1000,7,0.01,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-43.002819,-40.357070,...,-50.791760,1.908064,5,-49.178067,-48.591918,-48.679686,-48.177951,-48.495389,-48.624602,0.324536
6,1420.350065,114.645444,0.381436,0.127710,1000,12,0.3,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-39.546177,-37.920896,...,-39.862395,1.926300,1,-0.891498,-0.794153,-0.871579,-0.780114,-0.874893,-0.842447,0.045880
7,604.559071,77.514040,0.227717,0.059927,1000,7,0.01,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-43.002819,-40.357070,...,-50.791760,1.908064,5,-49.178067,-48.591918,-48.679686,-48.177951,-48.495389,-48.624602,0.324536
8,8.004952,0.291021,0.076866,0.017453,10,7,0.03,"{'xgbregressor__n_estimators': 10, 'xgbregress...",-82.025184,-79.499802,...,-72.783904,0.382726,9,-72.951012,-72.669547,-72.745037,-72.659358,-72.711616,-72.747314,0.106354
9,67.132393,1.826762,0.104334,0.014057,100,7,0.3,"{'xgbregressor__n_estimators': 100, 'xgbregres...",-41.055863,-39.710301,...,-46.902035,1.801798,4,-41.711130,-40.962670,-40.828219,-40.696486,-40.738046,-40.987310,0.373241


In [ ]:
# pipe_final_xgb = make_pipeline(
#     feature_set.column_transformer, 
#     XGBRegressor(
#         n_estimators=1000,
#         max_depth=7,
#         eta=0.3
#     )
# )

In [38]:
# HyperParam Tune CatBoost Regressor
params_catb = {
    "catboostregressor__iterations": [50, 100, 150],
    "catboostregressor__learning_rate": [0.01, 0.05, 0.1],
    "catboostregressor__depth": [4, 8, 12]
}

In [40]:
random_search_cat = RandomizedSearchCV(
    pipe_catboost,
    params_catb,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [41]:
random_search_cat.fit(feature_set.X_train_raw, feature_set.y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                                                'ELEVATION',

In [42]:
random_search_cat.best_params_

{'catboostregressor__learning_rate': 0.1,
 'catboostregressor__iterations': 50,
 'catboostregressor__depth': 12}

In [43]:
random_search_cat.best_score_

0.4307912380494714

In [65]:
# Stacking Regressor

models_selected = {
    "Random_Forest_reg": pipe_rf,
    "xgb_reg": pipe_final_xgb,
    "cat_boost": pipe_catboost

}

In [66]:
# stacking_results = {}


In [67]:
stacking_model = StackingRegressor(list(models_selected.items()))

In [83]:
_, results["Stacking"] = cross_val_scores(
    stacking_model,
    feature_set.X_train_raw, 
    feature_set.y_train,
    feature_set.X_val_raw,
    feature_set.y_val,
    return_train_score=True
    )

In [85]:
pd.DataFrame(results).T

,r2_val,mse_val,mape_val,r2_train,mse_train,mape_train
Ridge,0.385052,1909.311986,0.515557,0.356391,2204.814131,0.592824
Random Forest,0.544429,1414.471916,0.375215,0.937019,215.755881,0.158359
XGB,0.544609,1413.912253,0.424664,0.668918,1134.188983,0.440948
LGBM,0.492902,1574.456126,0.453599,0.540934,1572.623749,0.495500
Cat Boost,0.530885,1456.525966,0.426800,0.635735,1247.862146,0.457658
pipe_averaging,0.554623,1382.822864,0.408749,0.766054,801.428597,0.361582
Stacking,0.560813,1363.604405,0.381680,0.860425,478.143615,0.263858


In [87]:
#Hyperparamtune StackingRegressor

params_sr = {
    'stackingregressor__final_estimator': [XGBRegressor(n_estimators=1000,max_depth=7,eta=0.3), 
                                          CatBoostRegressor(),
                                          LGBMRegressor()]
}

In [90]:
random_search_stacking = RandomizedSearchCV(
    stacking_model,
    params_sr,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [108]:
#random_search_stacking.fit(feature_set.X_train_raw, feature_set.y_train)

In [107]:
#random_search_stacking.best_score_

In [ ]:
random_search_stacking.best_params_

In [78]:
# Voting Classifier

pipe_averaging = VotingRegressor(
    list(models_selected.items())
)

_, results["pipe_averaging"] = cross_val_scores(
    pipe_averaging,
    feature_set.X_train_raw, 
    feature_set.y_train,
    feature_set.X_val_raw,
    feature_set.y_val,
    return_train_score=True
)

In [80]:
pd.DataFrame(results)

,Ridge,Random Forest,XGB,LGBM,Cat Boost,pipe_averaging
r2_val,0.385052,0.544429,0.544609,0.492902,0.530885,0.554623
mse_val,1909.311986,1414.471916,1413.912253,1574.456126,1456.525966,1382.822864
mape_val,0.515557,0.375215,0.424664,0.453599,0.426800,0.408749
r2_train,0.356391,0.937019,0.668918,0.540934,0.635735,0.766054
mse_train,2204.814131,215.755881,1134.188983,1572.623749,1247.862146,801.428597
mape_train,0.592824,0.158359,0.440948,0.495500,0.457658,0.361582


In [92]:
# Hyperparam Tune Average Ensemble
params_voting_reg = {'votingregressor__voting':['hard', 'soft'],
          'votingregressor__weights':[(1,1,1), (2,1,1), 
                     (1,2,1), (1,1,2),(2,2,1),(1,2,2),(2,1,2)]}

In [93]:
random_search_voting = RandomizedSearchCV(
    pipe_averaging,
    params_voting_reg,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [109]:
#random_search_voting.fit(feature_set.X_train_raw, feature_set.y_train)

In [ ]:
random_search_voting.best_params_

In [ ]:
random_search_voting.best_score_

In [ ]:
#feature_set.column_transformer.transform(feature_set.X_test_raw)

In [104]:
prediction = random_search.predict(feature_set.X_test_raw)

In [105]:
results_dict = {"id": feature_set.X_test_raw["id"],
               TARGET_COLUMN: prediction}

In [106]:
pd.DataFrame(results_dict).set_index("id").to_csv("submission.csv")

## FCNN


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
feature_set = Dataprep(train_path, test_path)
feature_set.engineer_features()
feature_set.preprocess()

In [ ]:
def linear_block(input_size, output_size):
    return nn.Sequential(
        nn.Linear(input_size, output_size),
        nn.ReLU(),
        nn.Dropout(0.2)
    )


class EnergyRegressor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            linear_block(input_size, 2 * input_size),
            linear_block(2 * input_size, 3 * input_size),
            linear_block(3 * input_size, input_size),
            nn.Linear(input_size, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 1)
        )

    def forward(self, X):
        X = X.to(device)
        X = self.layers(X)
        return X

In [ ]:
trainloader = DataLoader(TensorDataset(feature_set.X_train, feature_set.y_train), batch_size=1, shuffle=True)
validloader = DataLoader(TensorDataset(feature_set.X_val, feature_set.y_val), batch_size=1, shuffle=True)

In [ ]:
model = EnergyRegressor(feature_set.X_train.shape[1])
model.to(device)

In [ ]:
def r2score(predictions, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - predictions) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

def mse(predictions, targets):
    return torch.mean((predictions - targets) ** 2)
        
def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())


In [ ]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs):

    for epoch in range(epochs):
        # train_batch_mse = []
        # train_batch_rsme = []
        # train_batch_r2 = []
        # val_batch_mse = []
        # val_batch_rsme = []
        # val_batch_r2 = []
        labels = []
        preds = []
        labels_val = []
        preds_val = []

        for X, y in trainloader:
            optimizer.zero_grad()
            X = X.to(device)
            y = y.to(device)
            y_hat = model(X.type(torch.float32))
            loss = criterion(y_hat, y.type(torch.float32))
            loss = loss.to(device)
            loss.backward()
            optimizer.step()
            # mse_train = mse(y_hat, y.type(torch.float32))
            # rsme_train = rmse(y_hat, y.type(torch.float32))
            # r2_train = r2score(y_hat, y.type(torch.float32))
            # train_batch_mse.append(mse_train)
            # train_batch_rsme.append(rsme_train)
            # train_batch_r2.append(r2_train)
            labels.extend(y.tolist())
            preds.extend(y_hat.tolist())

        labels = torch.tensor(labels, dtype=torch.bfloat16)
        preds = torch.tensor(preds, dtype=torch.bfloat16)

        train_mse = mse(labels, preds)
        train_rmse = rmse(labels, preds)
        train_r2 = r2score(labels, preds)
        # train_mse = torch.sum(torch.Tensor(train_batch_mse)) / len(trainloader)
        # train_rsme = torch.sum(torch.Tensor(train_batch_rsme)) / len(trainloader)
        # train_r2 = torch.sum(torch.Tensor(train_batch_r2)) / len(trainloader)
        
        model.eval()
        with torch.no_grad():
            for X_valid, y_valid in validloader:
                X_valid = X_valid.to(device)
                y_valid = y_valid.to(device)
                y_hat_val = model(X_valid.type(torch.float32))
                # mse_val = mse(y_hat_val, y_valid.type(torch.float32))
                # rsme_val = rmse(y_hat_val, y_valid.type(torch.float32))
                # r2_val = r2score(y_hat_val, y_valid.type(torch.float32))
                # val_batch_mse.append(mse_val)
                # val_batch_rsme.append(rsme_val)
                # val_batch_r2.append(r2_val)
                labels_val.extend(y_valid.tolist())
                preds_val.extend(y_hat_val.tolist())

            labels_val = torch.tensor(labels_val, dtype=torch.bfloat16)
            preds_val = torch.tensor(preds_val, dtype=torch.bfloat16)

            val_mse = mse(labels_val, preds_val)
            val_rsme = rmse(labels_val, preds_val)
            val_r2 = r2score(labels_val, preds_val)
            # val_mse = torch.sum(torch.Tensor(val_batch_mse)) / len(validloader)
            # val_rsme = torch.sum(torch.Tensor(val_batch_rsme)) / len(validloader)
            # val_r2 = torch.sum(torch.Tensor(val_batch_r2)) / len(validloader) 
        
        model.train()

        print(f"Epoch {epoch + 1}: ")
        print(f"Train MSE: {train_mse}. Train RMSE: {train_rmse}, Train R2: {train_r2}.")
        print(f"Val MSE: {val_mse}, Val RMSE: {val_rsme}, Val R2: {val_r2}.")

    return model

In [ ]:
device

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = criterion.to(device)
# optimizer = optimizer.to(device)
trained_model = trainer(model, criterion, optimizer, trainloader, validloader, epochs=25)

In [ ]:
def predict(model, X):
    return model(X.type(torch.float32))

In [ ]:
predict(trained_model, feature_set.X_test)